## Pre-processing Transaction data for test dataset
In this notebook, the whole dataset merged from profile and transcript is used. The main purpose is to assign offer ids to transacted data and then use them in the lambda function to predict if the given offer will be **successful** for the new customers.

The flow of the notebook is as follows:
* Carefully assign null values in the order of **reward -> event -> offer_id** in ***profile_transcript.csv*** dataset
* Merge the dataset with **portfolio** dataset
* Clean the dataset and perform feature engineering
* Split the dataset into train and test, then evaluate with Random Forest
* Export to csv and use in lambda function 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sagemaker
warnings.filterwarnings('ignore')
import json
import boto3

In [2]:
s3 = boto3.client('s3')
# session and role
sagemaker_session = sagemaker.Session()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()
key = 'input_data/profile_transcript.csv'
data_location = 's3://{}/{}'.format(bucket,key)
df = pd.read_csv(data_location)

In [3]:
portfolio = pd.read_json('portfolio.json', orient='records', lines=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272388 entries, 0 to 272387
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   gender            272388 non-null  object 
 1   age               272388 non-null  int64  
 2   member_id         272388 non-null  int64  
 3   became_member_on  272388 non-null  object 
 4   income            272388 non-null  float64
 5   membership_days   272388 non-null  int64  
 6   event             272388 non-null  object 
 7   time              272388 non-null  int64  
 8   amount            123957 non-null  float64
 9   reward            32070 non-null   float64
 10  offer_id          148431 non-null  float64
dtypes: float64(4), int64(4), object(3)
memory usage: 22.9+ MB


In [5]:
#Converting gender dtype to int
gender_list = {'M':0, 'F':1, 'O':2}
df.gender = df.gender.map(gender_list)
#became_member_on - as it is replaced as membership_days. Also, dropping amount for not in use
df1 = df.drop(['became_member_on', 'amount'], axis=1)

In [6]:
df1.isnull().sum()

gender                  0
age                     0
member_id               0
income                  0
membership_days         0
event                   0
time                    0
reward             240318
offer_id           123957
dtype: int64

In [7]:
df1.duplicated().sum()

0

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272388 entries, 0 to 272387
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   gender           272388 non-null  int64  
 1   age              272388 non-null  int64  
 2   member_id        272388 non-null  int64  
 3   income           272388 non-null  float64
 4   membership_days  272388 non-null  int64  
 5   event            272388 non-null  object 
 6   time             272388 non-null  int64  
 7   reward           32070 non-null   float64
 8   offer_id         148431 non-null  float64
dtypes: float64(3), int64(5), object(1)
memory usage: 18.7+ MB


In [9]:
df1.head()

,gender,age,member_id,income,membership_days,event,time,reward,offer_id
0,1,55,7577,112000.0,1653,transaction,18,NaN,NaN
1,1,55,7577,112000.0,1653,transaction,144,NaN,NaN
2,1,55,7577,112000.0,1653,offer received,408,NaN,4.0
3,1,55,7577,112000.0,1653,offer received,504,NaN,3.0
4,1,55,7577,112000.0,1653,transaction,528,NaN,NaN


***Getting reward values***

In [10]:
#Drop offer_id column for now
df_reward = df1.drop(['offer_id'], axis=1)

In [11]:
#Creating a new df for evaluating records with rewards
with_reward = df_reward[df_reward.event != 'transaction']

In [12]:
with_reward[with_reward.reward.isnull()].event.unique()

array(['offer received', 'offer viewed'], dtype=object)

In [13]:
##Filling null values in reward column to 0 as those records had event values of either 'offer received' or 'offer viewed'. As per the dataset, the reward will be given only if the event is 'offer completed'
with_reward.reward.fillna(0, inplace=True)

In [14]:
#Dropping the event column for analysis
with_reward.drop(['event'], axis=1, inplace=True)

In [15]:
with_reward.isnull().sum()

gender             0
age                0
member_id          0
income             0
membership_days    0
time               0
reward             0
dtype: int64

In [16]:
with_reward.reward.unique()

array([ 0.,  5., 10.,  2.,  3.])

In [17]:
with_reward.duplicated().sum()

11263

In [18]:
#Removing duplicates and evaluating the records
with_reward1 = with_reward.drop_duplicates(keep='first')
X_reward = with_reward1.drop(['reward'], axis=1)
y_reward = with_reward1.reward

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X_reward, y_reward, test_size=0.2, random_state=42)
rfc_reward = RandomForestClassifier(n_estimators=100, random_state=0)
rfc_reward.fit(X_train, y_train)
accuracy_score(y_test, rfc_reward.predict(X_test))

0.7086097543194576

In [20]:
df_reward.head()

,gender,age,member_id,income,membership_days,event,time,reward
0,1,55,7577,112000.0,1653,transaction,18,NaN
1,1,55,7577,112000.0,1653,transaction,144,NaN
2,1,55,7577,112000.0,1653,offer received,408,NaN
3,1,55,7577,112000.0,1653,offer received,504,NaN
4,1,55,7577,112000.0,1653,transaction,528,NaN


In [21]:
#Lets assign rewards to the records with event values of 'transaction'
without_reward = df_reward[df_reward.event == 'transaction']

In [22]:
rfc = RandomForestClassifier(n_estimators=100, random_state=0)
rfc.fit(X_reward, y_reward)

RandomForestClassifier(random_state=0)

In [23]:
#Drop reward column and assign the values with predictions. Event is dropped as well as the classifier excepts no event column
without_reward.drop(['reward','event'], axis=1, inplace=True)
without_reward['reward'] = rfc.predict(without_reward)

In [24]:
without_reward.head()

,gender,age,member_id,income,membership_days,time,reward
0,1,55,7577,112000.0,1653,18,0.0
1,1,55,7577,112000.0,1653,144,0.0
4,1,55,7577,112000.0,1653,528,5.0
8,1,75,14394,100000.0,1720,132,5.0
10,1,75,14394,100000.0,1720,144,5.0


In [25]:
print("df_reward shape: ", df_reward.shape)
print("with_reward shape: ", with_reward.shape)
print("without_reward shape: ", without_reward.shape)

df_reward shape:  (272388, 8)
with_reward shape:  (148431, 7)
without_reward shape:  (123957, 7)


In [26]:
#Creating a new df
df_reward_cleaned = with_reward.append(without_reward)
df_reward_cleaned.shape

(272388, 7)

In [27]:
df_reward.head()

,gender,age,member_id,income,membership_days,event,time,reward
0,1,55,7577,112000.0,1653,transaction,18,NaN
1,1,55,7577,112000.0,1653,transaction,144,NaN
2,1,55,7577,112000.0,1653,offer received,408,NaN
3,1,55,7577,112000.0,1653,offer received,504,NaN
4,1,55,7577,112000.0,1653,transaction,528,NaN


In [28]:
df_reward_cleaned.head()

,gender,age,member_id,income,membership_days,time,reward
2,1,55,7577,112000.0,1653,408,0.0
3,1,55,7577,112000.0,1653,504,0.0
5,1,55,7577,112000.0,1653,528,5.0
6,1,75,14394,100000.0,1720,0,0.0
7,1,75,14394,100000.0,1720,6,0.0


In [29]:
df_reward.shape

(272388, 8)

In [30]:
#Sorting the cleaned df to match the events
df_reward_cleaned.sort_index(inplace=True)

In [31]:
df_reward_cleaned['event'] = df_reward['event']
df_reward_cleaned.head()

,gender,age,member_id,income,membership_days,time,reward,event
0,1,55,7577,112000.0,1653,18,0.0,transaction
1,1,55,7577,112000.0,1653,144,0.0,transaction
2,1,55,7577,112000.0,1653,408,0.0,offer received
3,1,55,7577,112000.0,1653,504,0.0,offer received
4,1,55,7577,112000.0,1653,528,5.0,transaction


In [32]:
#Converting reward dtype to int
df_reward_cleaned.reward = df_reward_cleaned.reward.astype(int)

**Getting Event Values**
* The transaction data is given a dummy event with proper evaluations.

In [33]:
df_event1 = df_reward_cleaned.copy()
for i in df_event1.index:
    if df_event1.loc[i, 'event'] == 'transaction':
        if df_event1['reward'][i] != 0:
            df_event1.loc[i, 'event'] = 'offer completed'
        else:
            df_event1.loc[i, 'event'] = 'offer viewed'

In [34]:
df_event1.event.unique()

array(['offer viewed', 'offer received', 'offer completed'], dtype=object)

In [35]:
df_event1.head()

,gender,age,member_id,income,membership_days,time,reward,event
0,1,55,7577,112000.0,1653,18,0,offer viewed
1,1,55,7577,112000.0,1653,144,0,offer viewed
2,1,55,7577,112000.0,1653,408,0,offer received
3,1,55,7577,112000.0,1653,504,0,offer received
4,1,55,7577,112000.0,1653,528,5,offer completed


In [36]:
df_event1.shape

(272388, 8)

In [37]:
df1.head()

,gender,age,member_id,income,membership_days,event,time,reward,offer_id
0,1,55,7577,112000.0,1653,transaction,18,NaN,NaN
1,1,55,7577,112000.0,1653,transaction,144,NaN,NaN
2,1,55,7577,112000.0,1653,offer received,408,NaN,4.0
3,1,55,7577,112000.0,1653,offer received,504,NaN,3.0
4,1,55,7577,112000.0,1653,transaction,528,NaN,NaN


In [38]:
df1.shape

(272388, 9)

**Getting Offer Ids**

In [39]:
df_offer_id = df_event1.copy()
df_offer_id['offer_id'] = df1['offer_id']

In [40]:
df_offer_id.head()

,gender,age,member_id,income,membership_days,time,reward,event,offer_id
0,1,55,7577,112000.0,1653,18,0,offer viewed,NaN
1,1,55,7577,112000.0,1653,144,0,offer viewed,NaN
2,1,55,7577,112000.0,1653,408,0,offer received,4.0
3,1,55,7577,112000.0,1653,504,0,offer received,3.0
4,1,55,7577,112000.0,1653,528,5,offer completed,NaN


In [41]:
df_offer_id.offer_id.unique()

array([nan,  4.,  3.,  8.,  1.,  9., 10.,  7.,  6.,  5.,  2.])

In [42]:
#assigning 0 to null values in offer_id column
df_offer_id.offer_id.fillna(0, inplace=True)
#converting offer_id dtype to int
df_offer_id.offer_id = df_offer_id.offer_id.astype(int)

In [43]:
with_offer_id = df_offer_id[df_offer_id.offer_id != 0]
with_offer_id.duplicated().sum()

0

In [44]:
with_offer_id.shape

(148431, 9)

In [45]:
event_list = {'offer received':0, 'offer viewed':1, 'offer completed': 2}
with_offer_id.event = with_offer_id.event.map(event_list)

In [46]:
#Evaluating the records with offer_id
X_offer = with_offer_id.drop(['offer_id'], axis=1)
y_offer = with_offer_id.event
X_train, X_test, y_train, y_test = train_test_split(X_offer, y_offer, test_size=0.2, random_state=42)
rfc_offer = RandomForestClassifier(n_estimators=100, random_state=0)
rfc_offer.fit(X_train, y_train)
accuracy_score(y_test, rfc_offer.predict(X_test))

1.0

In [47]:
#Assigning offer ids
rfc_offer_full = RandomForestClassifier(n_estimators=100, random_state=0)
rfc_offer_full.fit(X_offer, y_offer)

RandomForestClassifier(random_state=0)

In [48]:
without_offer_id = df_offer_id[df_offer_id.offer_id == 0]
without_offer_id.drop(['offer_id'], axis=1, inplace=True)
event_list = {'offer received':0, 'offer viewed':1, 'offer completed': 2}
without_offer_id.event = without_offer_id.event.map(event_list)
without_offer_id['offer_id'] = rfc_offer_full.predict(without_offer_id)

In [49]:
#Merging the records with offer_id
df_offer_id_cleaned = with_offer_id.append(without_offer_id)
df_offer_id_cleaned.shape

(272388, 9)

Checking if the assigned offer_id has correct rewards. If not, manually assigning them.
* offer_id - 1, 2 reward - 10
* offer_id - 3, 8 reward - 0
* offer_id - 4, 5, 9 reward - 5
* offer_id - 6 reward - 3
* offer_id - 7, 10 reward - 2

In [50]:
df.shape

(272388, 11)

In [51]:
df_offer_id_cleaned[df_offer_id_cleaned.offer_id == 2].reward.unique()

array([ 0, 10,  5,  2,  3])

In [52]:
#Assign reward 10 to completed (2) offer id (2)
for i in df_offer_id_cleaned.index:
    if df_offer_id_cleaned.loc[i, 'offer_id'] == 2:
        if df_offer_id_cleaned['event'][i] == 2:
            df_offer_id_cleaned.loc[i, 'reward'] = 10

In [53]:
#After cleaning the records
df_offer_id_cleaned[df_offer_id_cleaned.offer_id == 2].reward.unique()

array([ 0, 10])

In [54]:
#To know the original event values
df_offer_id_cleaned['event_original'] = df['event']
df_offer_id_cleaned[df_offer_id_cleaned['event_original'] == 'transaction']

,gender,age,member_id,income,membership_days,time,reward,event,offer_id,event_original
0,1,55,7577,112000.0,1653,18,0,1,1,transaction
1,1,55,7577,112000.0,1653,144,0,1,1,transaction
4,1,55,7577,112000.0,1653,528,10,2,2,transaction
8,1,75,14394,100000.0,1720,132,10,2,2,transaction
10,1,75,14394,100000.0,1720,144,10,2,2,transaction
...,...,...,...,...,...,...,...,...,...,...
272378,1,62,474,82000.0,1646,84,10,2,2,transaction
272379,1,62,474,82000.0,1646,96,10,2,2,transaction
272382,1,62,474,82000.0,1646,480,10,2,2,transaction
272384,1,62,474,82000.0,1646,486,10,2,2,transaction


**Merging the transformed dataframe with portfolio dataframe**

In [55]:
df_portfolio = portfolio.copy()

In [56]:
channels = df_portfolio.channels.explode().unique()
channels

array(['email', 'mobile', 'social', 'web'], dtype=object)

In [57]:
# Create separate column for each channel
for channel in channels:
    df_portfolio.loc[:, channel] = df_portfolio.loc[:, "channels"].apply(lambda x: channel in x).astype(int)

In [58]:
offer_id_list = {
    'ae264e3637204a6fb9bb56bc8210ddfd': 1,
    '4d5c57ea9a6940dd891ad53e9dbe8da0': 2,
    '3f207df678b143eea3cee63160fa8bed': 3,
    '9b98b8c7a33c4b65b9aebfe6a799e6d9': 4,
    '0b1e1539f2cc45b7b9fa7c272da2e1d7': 5,
    '2298d6c36e964ae4a3e7e9706d1fb8c2': 6,
    'fafdcd668e3743c1bb461111dcafc2a4': 7,
    '5a8bc65990b245e5a138643cd4eb9837': 8,
    'f19421c1d4aa40978ebb69ca19b0e20d': 9,
    '2906b810c7d4411798c6938adc9daaa5': 10,
}

In [59]:
#Drop the channels column
df_portfolio.drop(columns=['channels'], inplace=True)
#Rename the id column to offer_id
df_portfolio.rename(columns={'id':'offer_id'}, inplace=True)
#Convert offer_id to integers
df_portfolio.loc[:,'offer_id'] = df_portfolio.loc[:,'offer_id'].map(offer_id_list)

In [60]:
df_portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   difficulty  10 non-null     int64 
 2   duration    10 non-null     int64 
 3   offer_type  10 non-null     object
 4   offer_id    10 non-null     int64 
 5   email       10 non-null     int64 
 6   mobile      10 non-null     int64 
 7   social      10 non-null     int64 
 8   web         10 non-null     int64 
dtypes: int64(8), object(1)
memory usage: 848.0+ bytes


In [61]:
df_portfolio.shape

(10, 9)

In [62]:
df_offer_id_cleaned.shape

(272388, 10)

In [63]:
df_merged = pd.merge(df_portfolio,df_offer_id_cleaned,on='offer_id', how='left')

In [64]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272388 entries, 0 to 272387
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   reward_x         272388 non-null  int64  
 1   difficulty       272388 non-null  int64  
 2   duration         272388 non-null  int64  
 3   offer_type       272388 non-null  object 
 4   offer_id         272388 non-null  int64  
 5   email            272388 non-null  int64  
 6   mobile           272388 non-null  int64  
 7   social           272388 non-null  int64  
 8   web              272388 non-null  int64  
 9   gender           272388 non-null  int64  
 10  age              272388 non-null  int64  
 11  member_id        272388 non-null  int64  
 12  income           272388 non-null  float64
 13  membership_days  272388 non-null  int64  
 14  time             272388 non-null  int64  
 15  reward_y         272388 non-null  int64  
 16  event            272388 non-null  int6

In [65]:
df_merged.duplicated().sum()

0

In [66]:
df_merged.head()

,reward_x,difficulty,duration,offer_type,offer_id,email,mobile,social,web,gender,age,member_id,income,membership_days,time,reward_y,event,event_original
0,10,10,7,bogo,1,1,1,1,0,1,75,14394,100000.0,1720,408,0,0,offer received
1,10,10,7,bogo,1,1,1,1,0,1,75,14394,100000.0,1720,408,0,1,offer viewed
2,10,10,7,bogo,1,1,1,1,0,1,75,14394,100000.0,1720,510,10,2,offer completed
3,10,10,7,bogo,1,1,1,1,0,1,61,7478,57000.0,1595,408,0,0,offer received
4,10,10,7,bogo,1,1,1,1,0,1,61,7478,57000.0,1595,426,10,2,offer completed


In [67]:
df_merged1 = df_merged.copy()
# offer_type - nominal encoding (0 - BOGO, 1 - Discount, 2 - Informational)
offer_types = {'bogo':0, 'discount':1, 'informational':2}
df_merged1.offer_type = df_merged1.offer_type.map(offer_types)
# Create a new column as 'offer_successful' and assign 1 for 'offer completed' and 0 for 'offer viewed' in event column
df_merged1['offer_successful'] = df_merged1.event.apply(lambda x: 1 if x==2 else 0)

**Drop below columns**
1. email - removing this from the feature list as email is used for all the offers
2. reward_x - as it is a general reward info from portfolio dataset. reward_y is the reward info for the offer.
3. event column - processed as 'offer_successful' (target) column
4. event_original column - Needed only for tracking the transactions. Not needed for the model

In [68]:
df_merged1.drop(['email', 'reward_x', 'event', 'event_original'], axis=1, inplace=True)
df_merged1.rename(columns={'reward_y':'reward'}, inplace=True)

In [69]:
df_merged1.head()

,difficulty,duration,offer_type,offer_id,mobile,social,web,gender,age,member_id,income,membership_days,time,reward,offer_successful
0,10,7,0,1,1,1,0,1,75,14394,100000.0,1720,408,0,0
1,10,7,0,1,1,1,0,1,75,14394,100000.0,1720,408,0,0
2,10,7,0,1,1,1,0,1,75,14394,100000.0,1720,510,10,1
3,10,7,0,1,1,1,0,1,61,7478,57000.0,1595,408,0,0
4,10,7,0,1,1,1,0,1,61,7478,57000.0,1595,426,10,1


In [70]:
df_merged1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272388 entries, 0 to 272387
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   difficulty        272388 non-null  int64  
 1   duration          272388 non-null  int64  
 2   offer_type        272388 non-null  int64  
 3   offer_id          272388 non-null  int64  
 4   mobile            272388 non-null  int64  
 5   social            272388 non-null  int64  
 6   web               272388 non-null  int64  
 7   gender            272388 non-null  int64  
 8   age               272388 non-null  int64  
 9   member_id         272388 non-null  int64  
 10  income            272388 non-null  float64
 11  membership_days   272388 non-null  int64  
 12  time              272388 non-null  int64  
 13  reward            272388 non-null  int64  
 14  offer_successful  272388 non-null  int64  
dtypes: float64(1), int64(14)
memory usage: 33.3 MB


In [71]:
X=df_merged1.drop(['offer_successful'], axis=1)
y=df_merged1.offer_successful

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=0)
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)
print("Accuracy Score: ", accuracy_score(y_test, rfc_pred))

In [ ]:
y_train.unique()

In [ ]:
#Need transaction set for evaluating in lambda function
df_whole = df_merged1.copy()
df_whole['event_original'] = df_merged['event_original']

In [ ]:
df_whole.duplicated().sum()

In [ ]:
df_whole.head()

In [ ]:
df_transact = df_whole[df_whole.event_original == 'transaction']
df_transact.drop(['event_original','offer_successful'], axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
df_transact_scaled = scale.fit_transform(df_transact)
np.savetxt('test_data.csv', df_transact_scaled, delimiter=',')

In [ ]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

In [ ]:
#Uploading the scaled data to test_data folder in the s3 default bucket
folder_name = "test_data"
test_data_path = 's3://{}/{}'.format(bucket, folder_name)
s3.upload_file('test_data.csv', bucket, folder_name+'/test_data.csv')